<h1 style='color: blue; font-size: 34px; font-weight: bold;'> Análise de Risco de Crédito direcionada por Modelagem Matemática e Estatística
</h1>
<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Este Notebook concentra código fonte da Tese de Bacharelado do curso de Engenharia de Controle e Automação da UNESP Sorocaba desenvolvido pelo aluno Leonardo Aderaldo Vargas.  </i> 
</p>  

# <font color='red' style='font-size: 40px;'> Problemática </font>
<hr style='border: 2px solid red;'>

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Muitas pessoas lutam para obter empréstimos devido a históricos de crédito insuficientes ou inexistentes. E, infelizmente, essa população é frequentemente aproveitada por credores não confiáveis.</i>
</p>  

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i>O Crédito Habitação busca ampliar a inclusão financeira para a população desbancarizada, proporcionando uma experiência de empréstimo positiva e segura. Para garantir que essa população carente tenha uma experiência de empréstimo positiva, o Home Credit utiliza uma variedade de dados alternativos - incluindo informações de telecomunicações e transações - para prever a capacidade de pagamento de seus clientes.</i>
</p>  
<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i>Dada a situação, o desafio é ajudá-los a desbloquear todo o potencial de seus dados. Isso garantirá que os clientes capazes de pagar não sejam rejeitados e que os empréstimos sejam concedidos com um principal, vencimento e calendário de pagamento que capacitará seus clientes a serem bem-sucedidos.</i>
</p>  




# <font color='red' style='font-size: 40px;'> Library   </font>
<hr style='border: 2px solid red;'>

In [1]:
# Importação das Bibliotecas e Parâmetros de Otimização
from functions import *
import warnings
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.2)
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = '14'
plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Tira os números do formato de Notação Científica
np.set_printoptions(suppress=True) # Tira os números do formato de Notação Científica em Numpy Arrays
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # Retira Future Warnings

# Spark Session
from pyspark.sql import SparkSession, Row 
#from pyspark.sql.functions import * as F
import pyspark.sql.functions as F
spark = SparkSession.builder.getOrCreate()

# <font color='red' style='font-size: 40px;'> 1. Entendimento da Base de Dados </font>
<hr style='border: 2px solid red;'>

![Alt text](image.png)

### 1.1) Bureaus

In [3]:
bureau_balance = pd.read_parquet('./data/bureau_balance.parquet')
print(bureau_balance.shape)
display(bureau_balance.head())

bureau = pd.read_parquet('./data/bureau.parquet')
print(bureau.shape)
display(bureau.head())

(27299925, 3)


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


(1716428, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.00,-153.00,NaN,0,91323.00,0.00,NaN,0.00,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.00,NaN,NaN,0,225000.00,171342.00,NaN,0.00,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.00,NaN,NaN,0,464323.50,NaN,NaN,0.00,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.00,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.00,NaN,77674.50,0,2700000.00,NaN,NaN,0.00,Consumer credit,-21,NaN


# <font color='red' style='font-size: 40px;'> 2. Análise Exploratória de Dados - Foco em Análises Univariadas  </font>
<hr style='border: 2px solid red;'>

# <font color='green' style='font-size: 30px;'> 2.1) Bureaus </font>
<hr style='border: 2px solid green;'>

- Puxando os Dados em Pandas e Spark

In [6]:
df_bureau = pd.read_parquet('./data/bureau.parquet')
df_bureau_balance = pd.read_parquet('./data/bureau_balance.parquet')
df_bureaus = df_bureau.merge(df_bureau_balance, on = 'SK_ID_BUREAU', how = 'left')

# bureau = spark.read.option('header', 'True').parquet('../05_modelo_risco_de_credito/data/bureau.parquet')
# bureau_balance = spark.read.option('header', 'True').parquet('../05_modelo_risco_de_credito/data/bureau_balance.parquet')
# df_bureaus_spark = bureau.join(bureau_balance, on = 'SK_ID_BUREAU', how = 'left')

- Modelando o Status: Clientes os quais deram DPD (número de dias que um pagamento de um empréstimo está atrasado em relação à data de vencimento original) serão automaticamente taxados como BAD 

In [7]:
df_bureau_balance['STATUS_GOOD_BAD'] = np.where(df_bureau_balance['STATUS'].isin(['1', '2', '3', '4', '5']), 'BAD', 'GOOD')
df_bureau_balance = df_bureau_balance[['SK_ID_BUREAU', 'STATUS_GOOD_BAD']].drop_duplicates()
print(df_bureau_balance.shape[0])
df_bureau_balance.head()

920607


,SK_ID_BUREAU,STATUS_GOOD_BAD
0,5715448,GOOD
27,5715449,GOOD
39,5715451,GOOD
65,5715452,GOOD
98,5715453,GOOD


- Calculando a quantidade de créditos fechados, ativos e que deram default 

In [8]:
df_bureaus['CREDIT_ACTIVE_CLOSED'] = np.where(df_bureaus['CREDIT_ACTIVE'] == 'Closed', 1, 0)
df_bureaus['CREDIT_ACTIVE_ACTIVE'] = np.where(df_bureaus['CREDIT_ACTIVE'] == 'Active', 1, 0)
df_bureaus['CREDIT_ACTIVE_BAD'] = np.where((df_bureaus['CREDIT_ACTIVE'] == 'Sold') | (df_bureaus['CREDIT_ACTIVE'] == 'Bad debt'), 1, 0)

df_qtd_emprestimos_bureaus = df_bureaus.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().sort_values(by = 'SK_ID_BUREAU', ascending = False).rename({'SK_ID_BUREAU':'AMOUNT_OF_LOANS'}, axis = 1)
df_qtd_emprestimos_fechados = df_bureaus.groupby('SK_ID_CURR', as_index=False)['CREDIT_ACTIVE_CLOSED'].sum().sort_values(by = 'CREDIT_ACTIVE_CLOSED', ascending = False).rename({'CREDIT_ACTIVE_CLOSED':'AMOUNT_OF_CLOSED_LOANS'}, axis = 1)
df_qtd_emprestimos_ativos = df_bureaus.groupby('SK_ID_CURR', as_index=False)['CREDIT_ACTIVE_ACTIVE'].sum().sort_values(by = 'CREDIT_ACTIVE_ACTIVE', ascending = False).rename({'CREDIT_ACTIVE_ACTIVE':'AMOUNT_OF_ACTIVE_LOANS'}, axis = 1)
df_qtd_emprestimos_bad = df_bureaus.groupby('SK_ID_CURR', as_index=False)['CREDIT_ACTIVE_BAD'].sum().sort_values(by = 'CREDIT_ACTIVE_BAD', ascending = False).rename({'CREDIT_ACTIVE_BAD':'AMOUNT_OF_BAD_LOANS'}, axis = 1)

df_bureaus.drop(['CREDIT_ACTIVE', 'CREDIT_ACTIVE_CLOSED', 'CREDIT_ACTIVE_ACTIVE', 'CREDIT_ACTIVE_BAD'], axis = 1, inplace = True)

# df_bureaus = df_bureaus.merge(df_qtd_emprestimos_bureaus, on = 'SK_ID_CURR', how = 'left')
# df_bureaus = df_bureaus.merge(df_qtd_emprestimos_fechados, on = 'SK_ID_CURR', how = 'left')
# df_bureaus = df_bureaus.merge(df_qtd_emprestimos_ativos, on = 'SK_ID_CURR', how = 'left')
# df_bureaus = df_bureaus.merge(df_qtd_emprestimos_bad, on = 'SK_ID_CURR', how = 'left')
# df_bureaus.head()

- Calculando os dias antes da aplicação 

In [ ]:
df_bureaus[['SK_ID_CURR', 'DAYS_CREDIT_ENDDATE']].sort_values(by = ['SK_ID_CURR', 'DAYS_CREDIT_ENDDATE'], ascending = True).drop_duplicates()

,SK_ID_CURR,DAYS_CREDIT_ENDDATE
3795270,100001,-1329.00
3795241,100001,-514.00
3795182,100001,-492.00
3795211,100001,-179.00
3795343,100001,411.00
...,...,...
15392248,456255,523.00
15392300,456255,806.00
15392347,456255,1463.00
15392407,456255,8620.00
